In [1]:
import numpy as np
import pandas as pd

In [2]:
import glob
# merge all the weather data
weather_2016 = glob.glob('./output/*2016.csv')
weather_2016.remove('./output/weather_initial_2016.csv')
weather_2017 = glob.glob('./output/*2017.csv')
weather_2017.remove('./output/weather_initial_2017.csv')
df_weather_2016 = pd.read_csv('./output/low_hour_2016.csv')
weather_2016.remove('./output/low_hour_2016.csv')
for i in weather_2016:
    df_weather_2016 = df_weather_2016.merge(pd.read_csv(i), on = 'regionidcity_fill', how = 'left')
df_weather_2017 = pd.read_csv('./output/low_hour_2017.csv')
weather_2017.remove('./output/low_hour_2017.csv')
for i in weather_2017:
    df_weather_2017 = df_weather_2017.merge(pd.read_csv(i), on = 'regionidcity_fill', how = 'left')
df_weather_2016 = df_weather_2016.fillna(0)
df_weather_2017 = df_weather_2017.fillna(0)

In [3]:
# read train csv
df_train_2016 = pd.read_csv('./Resources/train_2016_v2.csv')
df_train_2017 = pd.read_csv('./Resources/train_2017.csv')
df_prop_2016 = pd.read_csv('./output/prop_2016_refilled.csv')
df_prop_2017 = pd.read_csv('./output/prop_2017_refilled.csv')
df_merge_2016 = pd.merge(df_train_2016, df_prop_2016, how = 'left', on = 'parcelid')
df_merge_2017 = pd.merge(df_train_2017, df_prop_2017, how = 'left', on = 'parcelid')
df_merge_no_2017 = df_merge_2017.loc[df_merge_2017['latitude'].isna(),['parcelid', 'logerror', 'transactiondate']]
df_merge_no_2017 = df_merge_no_2017.merge(df_prop_2016, how = 'left', on = 'parcelid')
df_merge_2017 = pd.concat([df_merge_2017.loc[df_merge_2017['latitude'].notna(),:], df_merge_no_2017], sort=True)

/local/home/ningzesun/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (32,34,48,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/local/home/ningzesun/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_merge_2016 = pd.merge(df_merge_2016, df_weather_2016, how = 'left', on = 'regionidcity_fill')
df_merge_2017 = pd.merge(df_merge_2017, df_weather_2017, how = 'left', on = 'regionidcity_fill')

In [5]:
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder

def zoningcode2int( df, target ):
    storenull = df[ target ].isnull()
    enc = LabelEncoder( )
    df[ target ] = df[ target ].astype( str )

    print('fit and transform')
    df[ target ]= enc.fit_transform( df[ target ].values )
    print( 'num of categories: ', enc.classes_.shape  )
    df.loc[ storenull, target ] = np.nan
    print('recover the nan value')
    return enc

def fillna_knn( df, base, target, fraction = 0.1, threshold = 10, n_neighbors = 5 ):
    assert isinstance( base , list ) or isinstance( base , np.ndarray ) and isinstance( target, str ) 
    whole = [ target ] + base
    
    miss = df[target].isnull()
    notmiss = ~miss 
    nummiss = miss.sum()
    
    enc = OneHotEncoder()
    X_target = df.loc[ notmiss, whole ].sample( frac = fraction )
    
    enc.fit( X_target[ target ].unique().reshape( (-1,1) ) )
    
    Y = enc.transform( X_target[ target ].values.reshape((-1,1)) ).toarray()
    X = X_target[ base  ]
    
    print( 'fitting' )
    n_neighbors = n_neighbors
    clf = neighbors.KNeighborsClassifier( n_neighbors, weights = 'uniform' )
    clf.fit( X, Y )
    
    print( 'the shape of active features: ' ,enc.active_features_.shape )
    
    print( 'predicting' )
    Z = clf.predict(df.loc[miss, base])
    
    numunperdicted = Z[:,0].sum()
    if numunperdicted / nummiss *100 < threshold :
        print( 'writing result to df' )    
        df.loc[ miss, target ]  = np.dot( Z , enc.active_features_ )
        print( 'num of unperdictable data: ', numunperdicted )
        return enc
    else:
        print( 'out of threshold: {}% > {}%'.format( numunperdicted / nummiss *100 , threshold ) )

In [6]:
import warnings
warnings.filterwarnings('ignore')
df_merge_no_2017 = df_merge_2017.loc[df_merge_2017['latitude'].isna(),:]
df_merge_2017 = df_merge_2017.loc[df_merge_2017['latitude'].notna(),:]
fillna_knn(df = df_merge_2016, base = ['latitude', 'longitude'], target = 'lotsizesquarefeet', fraction = 0.15)
fillna_knn(df = df_merge_2017, base = ['latitude', 'longitude'], target = 'lotsizesquarefeet', fraction = 0.15)
from sklearn.preprocessing import LabelEncoder
zoningcode2int(df = df_merge_2016, target = 'propertyzoningdesc')
fillna_knn(df = df_merge_2016, base = ['latitude', 'longitude'], target = 'propertyzoningdesc', fraction = 0.15)
zoningcode2int(df = df_merge_2017, target = 'propertyzoningdesc')
fillna_knn(df = df_merge_2017, base = ['latitude', 'longitude'], target = 'propertyzoningdesc', fraction = 0.15)
df_merge_2017 = pd.concat([df_merge_2017, df_merge_no_2017])

fitting
the shape of active features:  (7111,)
predicting
writing result to df
num of unperdictable data:  0.0
fitting
the shape of active features:  (6479,)
predicting
writing result to df
num of unperdictable data:  0.0
fit and transform
num of categories:  (1997,)
recover the nan value
fitting
the shape of active features:  (1071,)
predicting
writing result to df
num of unperdictable data:  0.0
fit and transform
num of categories:  (1908,)
recover the nan value
fitting
the shape of active features:  (985,)
predicting
writing result to df
num of unperdictable data:  0.0


In [9]:
df_merge = pd.concat([df_merge_2016, df_merge_2017]).reset_index()
df_merge['transactiondate'] = pd.to_datetime(df_merge['transactiondate'], format='%Y-%m-%d %H:%M:%S')
df_merge['month'] = df_merge['transactiondate'].dt.month
df_merge['year'] = df_merge['transactiondate'].dt.year
df_merge.drop(['transactiondate', 'index', 'parcelid'], axis = 1, inplace = True)
df_merge.taxdelinquencyflag = df_merge.taxdelinquencyflag.replace('Y', 1)
df_merge.hashottuborspa = df_merge.hashottuborspa.replace('True', 1)
df_merge.hashottuborspa = df_merge.hashottuborspa.replace('None', 0)
df_merge.taxdelinquencyflag = df_merge.taxdelinquencyflag.astype('int64')

In [10]:
df_merge.to_csv('./output/init_merge.csv', index = False)

In [11]:
df_merge_2016.to_csv('./output/init_2016.csv', index = False)
df_merge_2017.to_csv('./output/init_2017.csv', index = False)